<a href="https://colab.research.google.com/github/StephWolter/PeterbestSellers/blob/main/Predicting_NYT_Lists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from tensorflow.keras.utils import to_categorical

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
# Prevent Pandas from truncating DataFrames
pd.set_option('display.max_colwidth', None)

In [42]:
# Load data into pandas DataFrames
path = '/content/drive/MyDrive/GW/Project4_Model/nyt_data/' # EDIT ACCORDING TO YOUR PARTICULAR FILE STRUCTURE
results_df = pd.read_csv(path + 'results.csv')
lists_df = pd.read_csv(path + 'lists.csv')
books_df = pd.read_csv(path + 'books.csv', on_bad_lines='skip')

# Create a DataFrame containing both book and list (i.e., genre) information
books_lists = results_df[['list_id', 'book_id']].drop_duplicates()
bl_full = books_lists.merge(books_df, on='book_id')
bl_full.head()

,list_id,book_id,book_title,book_image,book_description,author_id,imprint_id
0,1,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1
1,3,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1
2,12,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1
3,5,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1
4,18,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1


In [43]:
# Review the list DataFrame in order to determine which lists are fiction and which are nonfiction
lists_df

,list_id,list_name
0,1,Combined Print and E-Book Fiction
1,2,Combined Print and E-Book Nonfiction
2,3,Hardcover Fiction
3,4,Hardcover Nonfiction
4,5,Trade Fiction Paperback
5,6,Paperback Nonfiction
6,7,Advice How-To and Miscellaneous
7,8,Childrens Middle Grade Hardcover
8,9,Picture Books
9,10,Series Books


In [44]:
# Create an array of list_id's that are fiction and another array of list_id's that are nonfiction
fiction_ids = [
    1,
    3,
    5,
    8,
    9,
    10,
    11,
    12,
    17,
    18,
    19,
    20,
    21,
    22,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    42,
    47,
    50,
    51,
    56,
    58,
    59
]
nonfiction_ids = [id for id in lists_df['list_id'] if id not in fiction_ids]

In [45]:
# Define a function that assigns a value of 1 to books on a fiction list and a value of 0 to books on a nonfiction list
def is_fiction(value):
  if value in fiction_ids:
    return 1
  else:
    return 0

# Use .map() to add the column, is_fiction, to the DataFrame
bl_full['is_fiction'] = bl_full['list_id'].map(is_fiction)
bl_full.head()

,list_id,book_id,book_title,book_image,book_description,author_id,imprint_id,is_fiction
0,1,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1
1,3,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1
2,12,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1
3,5,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1
4,18,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1


In [46]:
# Add a column, title_desc, which is simply a concatenation of the book title and the book description
bl_full['title_desc'] = bl_full['book_title'] + " " + bl_full['book_description']
bl_full.head()

,list_id,book_id,book_title,book_image,book_description,author_id,imprint_id,is_fiction,title_desc
0,1,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect."
1,3,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect."
2,12,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect."
3,5,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect."
4,18,1,WHERE THE CRAWDADS SING,https://storage.googleapis.com/du-prd/books/images/9780735219090.jpg,"In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1,1,1,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect."


In [47]:
# Create a new DataFrame, df, that pulls the title_desc and is_fiction columns from bl_full
df = bl_full[['title_desc', 'is_fiction']]

# Drop duplicates and NaN's
df = df.drop_duplicates()
df = df.dropna()
df.head()

,title_desc,is_fiction
0,"WHERE THE CRAWDADS SING In a quiet town on the North Carolina coast in 1969, a young woman who survived alone in the marsh becomes a murder suspect.",1
5,THE SAVIOR The 17th book in the Black Dagger Brotherhood series. Secrets about inhumane experiments surface at a biomedical firm.,1
7,"A GENTLEMAN IN MOSCOW A Russian count undergoes 30 years of house arrest in the Metropol hotel, across from the Kremlin.",1
10,SUPERMARKET A depressed young man’s secrets are exposed when the grocery store where he works becomes a crime scene.,1
12,THE CORNWALLS ARE GONE An Army intelligence officer must commit a crime or lose her kidnapped husband and daughter.,1


In [11]:
# Use regex to remove punctuation from the title_desc strings
desc_text = df['title_desc'].astype(str).str.replace('[^\w\s]','')

<ipython-input-11-d4d83208094a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  desc_text = df['title_desc'].astype(str).str.replace('[^\w\s]','')


In [12]:
# Tokenize the text. Tokenization refers to splitting strings into tokens (for example, splitting a sentence into individual words, by splitting on whitespace)
# See https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(desc_text)
X_text = tokenizer.texts_to_sequences(desc_text)

# Pad sequences to a fixed length of 350, adding 0s if the text is less than 350 characters, and truncating the text if it is greater than 350 characters
max_sequence_length = 350
desc_text_padded = pad_sequences(X_text, maxlen=max_sequence_length, padding='post')
X_text_padded = desc_text_padded

In [13]:
# Target array
y = df['is_fiction']

In [14]:
# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X_text_padded, y, random_state=42)

## **First Model Iteration**

In [19]:
# Create an empty sequential model
nn = tf.keras.models.Sequential()

# First layer
nn.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=X_train.shape[1]))

# Hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 200)               70200     
                                                                 
 dense_3 (Dense)             (None, 32)                6432      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 76,665
Trainable params: 76,665
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile and train the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.fit(X_train, y_train, epochs=15)

Epoch 1/15
253/253 [==============================] - 1s 2ms/step - loss: 42.9319 - accuracy: 0.5660
Epoch 2/15
253/253 [==============================] - 0s 2ms/step - loss: 0.9861 - accuracy: 0.6380
Epoch 3/15
253/253 [==============================] - 0s 2ms/step - loss: 0.7096 - accuracy: 0.6560
Epoch 4/15
253/253 [==============================] - 0s 2ms/step - loss: 0.6480 - accuracy: 0.6622
Epoch 5/15
253/253 [==============================] - 1s 2ms/step - loss: 0.6410 - accuracy: 0.6637
Epoch 6/15
253/253 [==============================] - 1s 2ms/step - loss: 0.6335 - accuracy: 0.6663
Epoch 7/15
253/253 [==============================] - 1s 3ms/step - loss: 0.6413 - accuracy: 0.6667
Epoch 8/15
253/253 [==============================] - 1s 3ms/step - loss: 0.6311 - accuracy: 0.6670
Epoch 9/15
253/253 [==============================] - 1s 3ms/step - loss: 0.6303 - accuracy: 0.6675
Epoch 10/15
253/253 [==============================] - 1s 2ms/step - loss: 0.6285 - accuracy: 0.667

In [21]:
# Display predictive accuracy on test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

85/85 - 1s - loss: 0.6536 - accuracy: 0.6720 - 656ms/epoch - 8ms/step
Loss: 0.6535598039627075, Accuracy: 0.6719970107078552


# **Second Model Iteration**

In [16]:
# Create an empty sequential model
nn = tf.keras.models.Sequential()

# Add an Embedding layer to take the integer-encoded title_descs and look up an embedding vector for each word-index
# Adapted from https://www.tensorflow.org/tutorials/keras/text_classification
nn.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_sequence_length))

# Add a Dropout layer to randomly set 0.2 of the input units to 0 at each step during training time. This helps prevent overfitting
# See https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
nn.add(tf.keras.layers.Dropout(0.2))

# Add a Global Average Pooling layer to return a fixed-length output vector for each example by averaging over the sequence dimension
nn.add(tf.keras.layers.GlobalAveragePooling1D())

# Hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 350, 50)           1051850   
                                                                 
 dropout (Dropout)           (None, 350, 50)           0         
                                                                 
 global_average_pooling1d (G  (None, 50)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                1632      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,053,515
Trainable params: 1,053,515
Non-trainable params: 0
______________________________________________

In [17]:
# Compile and train the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.fit(X_train, y_train, epochs=15)

Epoch 1/15
253/253 [==============================] - 5s 15ms/step - loss: 0.6430 - accuracy: 0.6628
Epoch 2/15
253/253 [==============================] - 4s 15ms/step - loss: 0.6281 - accuracy: 0.6662
Epoch 3/15
253/253 [==============================] - 5s 19ms/step - loss: 0.5869 - accuracy: 0.6675
Epoch 4/15
253/253 [==============================] - 4s 15ms/step - loss: 0.4386 - accuracy: 0.8153
Epoch 5/15
253/253 [==============================] - 4s 16ms/step - loss: 0.3249 - accuracy: 0.8774
Epoch 6/15
253/253 [==============================] - 6s 22ms/step - loss: 0.2646 - accuracy: 0.9067
Epoch 7/15
253/253 [==============================] - 4s 16ms/step - loss: 0.2302 - accuracy: 0.9167
Epoch 8/15
253/253 [==============================] - 4s 16ms/step - loss: 0.2054 - accuracy: 0.9267
Epoch 9/15
253/253 [==============================] - 5s 20ms/step - loss: 0.1858 - accuracy: 0.9347
Epoch 10/15
253/253 [==============================] - 4s 16ms/step - loss: 0.1707 - accura

In [18]:
# Display predictive accuracy on test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

85/85 - 0s - loss: 0.3112 - accuracy: 0.8773 - 332ms/epoch - 4ms/step
Loss: 0.31123673915863037, Accuracy: 0.8772777915000916
